**This portion processes the videos into frames to prepare for learning**

This code uses methods provided in this tutorial [here](https://www.analyticsvidhya.com/blog/2019/09/step-by-step-deep-learning-tutorial-video-classification-python/) in order to process videos and setting up the neural net. Credits go to Pulkit Sharma.

In [0]:
# Lots of headers, though some may be unused rn
import cv2
import math
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from keras.preprocessing import image
import numpy as np
from keras.utils import np_utils
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd drive/'My Drive'/Videos_DLP_v3

[Errno 2] No such file or directory: 'drive/My Drive/Videos_DLP_v3'
/content/drive/My Drive/Videos_DLP_v3


In [0]:
# Prepares stuff for training
# open the .txt file which have names of training videos
f = open("Training.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe using video names
train = pd.DataFrame()
train['video_name'] = videos
train = train[:-1]
train.head()

,video_name
0,Drumming_1.MP4
1,Drumming_2.MP4
2,Drumming_3.MP4
3,Drumming_4.MP4
4,Drumming_9.MP4


In [0]:
# Prepares stuff for testing
# open the .txt file which have names of test videos
f = open("Testing.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test.head()

,video_name
0,Drumming_5.MP4
1,Drumming_6.MP4
2,Drumming_7.MP4
3,Drumming_8.MP4
4,Drumming_15.MP4


In [0]:
# Tag creation here
# creating tags for training videos
train_video_tag = []
for i in range(train.shape[0]):
    train_video_tag.append(train['video_name'][i].split('/')[0])
    
train['tag'] = train_video_tag

# creating tags for test videos
test_video_tag = []
for i in range(test.shape[0]):
    test_video_tag.append(test['video_name'][i].split('/')[0])
    
test['tag'] = test_video_tag

In [0]:
# Breaks the video by frames for training set
for i in tqdm(range(train.shape[0])):
    count = 0
    videoFile = train['video_name'][i]
    cap = cv2.VideoCapture(videoFile.split(' ')[0])   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames in a new folder named train_1
            filename ='training/' + videoFile.split(' ')[0] +"_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()

100%|██████████| 69/69 [02:23<00:00,  2.08s/it]


In [0]:
# getting the names of all the images for training set
images = glob("training/*.jpg")
train_image = []
train_class = []
for i in tqdm(range(len(images))):
    # creating the image name
    train_image.append(images[i].split('/')[1])
    # creating the class of image
    vidclass = images[i].split('/')[1].split('_')[0]
    train_class.append(vidclass.lower())
    
# storing the images and their class in a dataframe
train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class 

# converting the dataframename into csv file 
print(train_data['class'])
train_data.to_csv('training/trains.csv',header=True, index=False)

100%|██████████| 505/505 [00:00<00:00, 169395.67it/s]

0      drumming
1      drumming
2      drumming
3      drumming
4      drumming
         ...   
500    drumming
501    drumming
502    drumming
503    drumming
504    drumming
Name: class, Length: 505, dtype: object


In [0]:
# Breaks the video by frames for testing set
for i in tqdm(range(test.shape[0])):
    count = 0
    videoFile = test['video_name'][i]
    cap = cv2.VideoCapture(videoFile.split(' ')[0])   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        # Save frames every second
        if (frameId % math.floor(frameRate) == 0):
            filename ='testing/' + videoFile.split(' ')[0] +"_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()

100%|██████████| 33/33 [01:05<00:00,  1.97s/it]


In [0]:
# getting the names of all the images for testing set
images = glob("testing/*.jpg")
test_image = []
test_class = []
for i in tqdm(range(len(images))):
    # creating the image name
    test_image.append(images[i].split('/')[1])
    # creating the class of image
    vidclass = images[i].split('/')[1].split('_')[0]
    test_class.append(vidclass.lower())
    
# storing the images and their class in a dataframe
test_data = pd.DataFrame()
test_data['image'] = test_image
test_data['class'] = test_class 

# converting the dataframename into csv file 
print(test_data['class'])
test_data.to_csv('testing/testings.csv',header=True, index=False)

100%|██████████| 255/255 [00:00<00:00, 212422.55it/s]

0      drumming
1      drumming
2      drumming
3      drumming
4      drumming
         ...   
250     erratic
251     erratic
252     erratic
253     erratic
254     erratic
Name: class, Length: 255, dtype: object


**Get ready to train the model**

In [0]:
import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [0]:
ls

Drumming_0.mp4   Drumming_5.MP4  Erratic_32.MOV  Pockets_21.MP4
Drumming_10.MP4  Drumming_6.MP4  Erratic_33.MOV  Pockets_22.mov
Drumming_11.MP4  Drumming_7.MP4  Erratic_3.MP4   Pockets_23.mov
Drumming_12.MP4  Drumming_8.MP4  Erratic_4.MP4   Pockets_24.mov
Drumming_13.MP4  Drumming_9.MP4  Erratic_5.MP4   Pockets_25.mov
Drumming_14.MP4  Erratic_0.mp4   Erratic_6.MP4   Pockets_26.MP4
Drumming_15.MP4  Erratic_10.MP4  Erratic_7.MP4   Pockets_27.mp4
Drumming_16.MP4  Erratic_11.MP4  Erratic_8.MP4   Pockets_28.mp4
Drumming_17.MP4  Erratic_12.MP4  Erratic_9.MP4   Pockets_29.mp4
Drumming_18.MP4  Erratic_13.MP4  Negative_1.MOV  Pockets_2.MP4
Drumming_19.MP4  Erratic_14.MP4  Negative_2.MOV  Pockets_30.MOV
Drumming_1.MP4   Erratic_15.MP4  Negative_3.MOV  Pockets_31.MOV
Drumming_20.MP4  Erratic_16.MP4  Negative_4.MOV  Pockets_32.MOV
Drumming_21.MOV  Erratic_17.MP4  Negative_5.MOV  Pockets_33.MOV
Drumming_22.MOV  Erratic_18.MP4  Negative_6.MOV  Pockets_3.MP4
Drumming_23.MOV  Erratic_19.MP4  Negative_

In [0]:
train = pd.read_csv('training/trains.csv')
train.head()
# Setting up the frames here
train_image = []

# Processes the images here.
for i in tqdm(range(train.shape[0])):
    img = image.load_img('training/'+train['image'][i], target_size=(224,224,3))
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
    
# convert to np array
X = np.array(train_image)

100%|██████████| 505/505 [00:11<00:00, 45.72it/s]


In [0]:
# Creating validation set
# separating the target
y = train['class']

# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [0]:
# Creating the base model of pre-trained VGG16 model
# This is one form of CNN that can be used on each frame individually 
base_model = VGG16(weights='imagenet', include_top=False)

In [0]:
X_train = base_model.predict(X_train) # extracting features for training frames
X_test = base_model.predict(X_test) # extracting features for testing frames

In [0]:
X_train.shape

(404, 7, 7, 512)

In [0]:
X_test.shape

(101, 7, 7, 512)

In [0]:
# Reshaping the training as well as validation frames
X_train = X_train.reshape(404, 7*7*512)
X_test = X_test.reshape(101, 7*7*512)

# Normalizing the pixel values
max = X_train.max()
X_train = X_train/max
X_test = X_test/max

In [0]:
# Using the sequential architecture
# Using three layers on the sequential model
# TODO: Change the dropout between the layers
# Sizes must also change
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.7))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

Training Time

In [0]:
# Compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [0]:
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight11.hdf5', save_best_only=True, monitor='val_loss', mode='min')

In [0]:
# training the model
model.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), callbacks=[mcp_save],  batch_size=64)

Train on 404 samples, validate on 101 samples
Epoch 1/15
404/404 [==============================] - 3s 6ms/step - loss: 0.0553 - accuracy: 0.9901 - val_loss: 0.1207 - val_accuracy: 0.9901
Epoch 2/15
404/404 [==============================] - 3s 6ms/step - loss: 0.0330 - accuracy: 0.9901 - val_loss: 0.1102 - val_accuracy: 0.9901
Epoch 3/15
404/404 [==============================] - 3s 6ms/step - loss: 0.1144 - accuracy: 0.9876 - val_loss: 0.0801 - val_accuracy: 0.9901
Epoch 4/15
404/404 [==============================] - 3s 6ms/step - loss: 0.0769 - accuracy: 0.9827 - val_loss: 0.0791 - val_accuracy: 0.9901
Epoch 5/15
404/404 [==============================] - 3s 6ms/step - loss: 0.0279 - accuracy: 0.9926 - val_loss: 0.1195 - val_accuracy: 0.9802
Epoch 6/15
404/404 [==============================] - 3s 6ms/step - loss: 0.0224 - accuracy: 0.9901 - val_loss: 0.1132 - val_accuracy: 0.9901
Epoch 7/15
404/404 [==============================] - 3s 8ms/step - loss: 0.0393 - accuracy: 0.9926 - 

**Time to test everything here**

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
from tqdm import tqdm
from keras.applications.vgg16 import VGG16
import cv2
import math
import os
from glob import glob
from scipy import stats as s

In [0]:
# getting the test list
f = open("Testing.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating the dataframe
test = pd.DataFrame()
test['video_name'] = videos
test = test[:-1]
test_videos = test['video_name']
test.head()

,video_name
0,Drumming_5.MP4
1,Drumming_6.MP4
2,Drumming_7.MP4
3,Drumming_8.MP4
4,Drumming_15.MP4


In [0]:
# creating the tags
train = pd.read_csv('training/trains.csv')
y = train['class']
y = pd.get_dummies(y)

In [0]:
# creating two lists to store predicted and actual tags
predict = []
actual = []

# for loop to extract frames from each test video
for i in tqdm(range(test_videos.shape[0])):
    count = 0
    videoFile = test_videos[i]
    cap = cv2.VideoCapture(videoFile.split(' ')[0])   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    # removing all other files from the temp folder
    # !rm -rf temp
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames of this particular video in temp folder
            filename ='temp/' + "_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()
    
    # reading all the frames from temp folder
    images = glob("temp/*.jpg")
    prediction_images = []
    for i in range(len(images)):
        img = image.load_img(images[i], target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img/255
        prediction_images.append(img)
    prediction_images = np.array(prediction_images)
    prediction_images = base_model.predict(prediction_images)
    prediction_images = prediction_images.reshape(prediction_images.shape[0], 7*7*512)
    prediction = model.predict_classes(prediction_images)
    # appending the mode of predictions in predict list to assign the tag to the video
    predict.append(y.columns.values[s.mode(prediction)[0][0]])

    # appending the actual tag of the video
    vidclass = (videoFile.split('_')[0])
    actual.append(vidclass.lower())
    

100%|██████████| 33/33 [03:53<00:00,  7.07s/it]


In [0]:
# 0: drumming
# 1: erratic
# 2: pockets
# checking the accuracy of the predicted tags
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100

90.9090909090909

The end!
